***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-1ff3  GCE       3                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import math
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 23 15:15 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'irproject-414719bucket' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and not b.name.startswith("postings_gcp"):
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("title", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [49]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb', 'became', 'may']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)
stemmer = PorterStemmer()

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

#####################################################################################################
stemDict = {}
l = ["genetics", "economic", "telecommunications", "computer", "bioinformatics"]

def word_title_count(title, id):
    containl = False
    tokens = [token.group() for token in RE_WORD.finditer(title.lower())]
    sizeOfTitle = 0
    word_counts = Counter()

    for token in tokens:
        if (token not in all_stopwords):
            if(token in stemDict):
                word_counts[token] += 1
            else:
                stemDict[token] = stemmer.stem(token)
                word_counts[stemDict[token]] += 1

            sizeOfTitle += 1
            if(containl == False):
                if(token in l):
                    containl = True

    if len(stemDict) >= 20000:
        stemDict.clear()

    if(containl == False and id != 0):
        return []

    return [(token, (id, count/sizeOfTitle)) for token, count in word_counts.items()]

#####################################################################################################

def reduce_word_title_counts(unsorted_pl):
    return sorted(unsorted_pl, key=lambda x: x[0])
#####################################################################################################




In [50]:
word_counts = doc_text_pairs.flatMap(lambda x: word_title_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_title_counts)   # Sort by document id and then merge by term.
postings_filtered = postings.filter(lambda x: len(x[1])>20)
postings_filtered = postings_filtered.collectAsMap()

In [51]:
inverted = InvertedIndex()
inverted.tfTitle = postings_filtered
inverted.write_index('postings_gcp_Title', 'indexTitle', bucket_name)

index_src = "indexTitle.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_Title/{index_src}'
!gsutil cp $index_src $index_dst

CommandException: No URLs matched: indexTitle.pkl


In [52]:
inverted_index = InvertedIndex.read_index('postings_gcp_Title', 'indexTitle',bucket_name)

In [54]:
simDoc = Counter()
query = stemmer.stem("genetics")

query = dict(word_title_count(query, 0))
for term,  value in query.items():
    docTfItf = inverted_index.tfTitle[term]
    for doc, weight in docTfItf:
        simDoc[doc] += value[1] * weight


top_10_similar_docs = simDoc.most_common(10)
top_10_similar_docs

[(12266, 1.0),
 (68300, 0.5),
 (155192, 0.5),
 (215038, 0.5),
 (297382, 0.5),
 (339838, 0.5),
 (418605, 0.5),
 (480107, 0.5),
 (553182, 0.5),
 (709360, 0.5)]